In [482]:
from pyspark import SparkContext
import time
import json
import sys
import math

# Task 1

## Case 1

In [483]:
sc = SparkContext.getOrCreate()

In [484]:
filepath = '../resource/asnlib/publicdata/small1.csv'
# filepath = '../resource/asnlib/publicdata/ta_feng_all_months_merged.csv'

In [506]:
case_number = 1

In [507]:
# Read File, skipping header
review = sc.textFile(filepath).zipWithIndex().filter(lambda x: x[1] > 0).map(lambda line: line[0].split(","))
if case_number == 1:
    review = review.map(lambda x: (x[0], x[1]))
elif case_number == 2:
    review = review.map(lambda x: (x[1], x[0]))

else:
    print("Invalid case number")
    exit(1)


In [508]:
# review.collect()

In [509]:
basket_rdd = review.groupByKey().mapValues(set).map(lambda x: (x[0], list(x[1])))

In [510]:
user_basket_memory = basket_rdd.collect()

**A-priori algorithm**

In [8]:
user_basket_memory[:5]


[('1', ['102', '100', '101', '98']),
 ('4', ['102', '97', '101', '99', '103']),
 ('8', ['102', '97', '104', '99', '103', '98']),
 ('9', ['99', '97', '98']),
 ('10', ['97', '98'])]

In [9]:
from collections import defaultdict

In [10]:
def hash_items(basket_rdd: list) -> dict:
    '''Translates item names into integers from 1 to n

    Args:
        basket_rdd (list): [(key, value), ...]

    Returns:
        (dict): {item: hash, ...}
    '''
    subset_items = set()
    for key, list_items in basket_rdd:
        subset_items = subset_items | set(list_items)
    subset_items = list(subset_items)
    n = range(1, len(subset_items) + 1) # n is the number of distinct items
    map_to_int = { item: n_item for item, n_item in zip(subset_items, n)}
    return map_to_int

In [11]:
# user_basket_memory

In [12]:
map_item_hash = hash_items(user_basket_memory)

In [13]:
user_basket_memory_hashed = []
for key, list_items in user_basket_memory:
    user_basket_memory_hashedlist = [map_item_hash[item] for item in list_items]
    user_basket_memory_key = key
    user_basket_memory_hashed.append((user_basket_memory_key, user_basket_memory_hashedlist))

In [14]:
def triangular_matrix_method():
    pass

In [15]:
import itertools

In [16]:
map_item_hash.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [17]:
# list(itertools.combinations(map_item_hash.values(), 2))

In [18]:
new_candidate = (1,2,3)

In [19]:
c = set(itertools.combinations(new_candidate, 2))

In [137]:
c = set([(1, 2)])

In [140]:
c.add((2,1))

In [127]:
c <= set([(2,3), (1,2), (1,3), (4,5)])

True

In [62]:
set((1,2) )

{1, 2}

In [63]:
L_1 = [(1,), (2,), (3,)]

In [71]:
{1} in set((1,2,3))

False

In [72]:
a = (1,)

In [74]:
a[0]

1

In [139]:
subset = (1,3)
#b,m

In [82]:
L_1 = [(1,), (3,), (4,), (2,)]
# b,c,j,m

In [319]:
# { {b,m} {b,c}  {c,m}  {c,j} }

In [83]:
L_k_1 = [(1,2), (1,3), (3,2), (3,4)]

In [129]:
L_k_1

[(1, 2), (1, 3), (3, 2), (3, 4)]

In [132]:
set([(1,2)]) <= set(L_k_1)

True

In [134]:
set([(2,1)]).intersection(set(L_k_1))

set()

In [101]:
L_k_1

[(1, 2), (1, 3), (3, 2), (3, 4)]

In [87]:
L_1

[(1,), (3,), (4,), (2,)]

In [150]:
subset

(1, 3)

In [76]:
# a = new_candidate(subset, L_k_1, L_1, 3)

In [320]:
subset = (1,)
L_1 = [(1,), (3,), (4,), (2,)]
L_k_1 = [(1,), (3,), (4,), (2,)]
# b,c,j,m
#b,m

In [64]:
# a = new_candidate(subset, L_k_1, L_1, 2)

In [193]:
a = set()

In [161]:
a.add({(2,1), {2,2}})

TypeError: unhashable type: 'set'

In [159]:
L_k_1.update([(3,2)])

In [164]:
L_k_1 | {(3,3), (3,2)}

{(1,), (2,), (2, 3), (3,), (3, 2), (3, 3), (5,), (7,), (8,), (11,), (12,)}

# A-priori Code

In [198]:
def new_candidate(subset: tuple, L_k_1: set, L_1: set, k: int, C_k: list) -> set:
    list_of_candidates = set()
    for frequent_item in L_1: # Iterate over each item of L_1 that does not intersect
        if frequent_item[0] in subset: # Do nothing if L_1 item is already in subset
            continue
        # For each basket, we need only look at those items that are in L_1
        new_candidate = set(sorted(subset + frequent_item)) 
        
        # print(new_candidate)
        # print(list_of_candidates)
        if new_candidate not in list_of_candidates:
            # Examine each pair and determine whether or not that pair is in L_k - 1
            subset_combination = set(itertools.combinations(new_candidate, k-1))
            # print(subset_combination)
            # print(subset_combination)
            if subset_combination <= L_k_1:
                list_of_candidates.add(tuple(new_candidate)) #if new_candidate not in C_k else list_of_candidates # Remove duplicates
    return list_of_candidates

In [199]:
def generate_candidates(L_k_1: set, L_1: set, k: int) -> set:
    # Generatin C_k from L_k_1 and L_1
    C_k = set()
    for subset in L_k_1:
        new_subsets_candidates = new_candidate(subset, L_k_1, L_1, k, C_k)
        for new_subset in new_subsets_candidates:
            C_k.add(new_subset) #if new_subsets_candidates not in C_k else C_k
            # break

    return C_k

In [202]:
def count_itemsets_of_size_k(basket_rdd: list, candidates_list: list, threshold: int) -> set:
    '''
    Count k-size candidate itemsets and returns those that are frequent

    Args:
        basket_rdd (list)
        candidates_list (list)
        threshold (int)
    Returns:
        
    '''
    candidates_count = defaultdict(int)
    for _, item_list in basket_rdd:
        for candidate in candidates_list:
            if set(candidate) <= set(item_list):
                candidates_count[candidate] += 1 # increase 1 count

    return {k for k,v in candidates_count.items() if v >= threshold}
            

In [513]:
# def hash_items(basket_rdd: list) -> dict:
#     '''Translates item names into integers from 1 to n

#     Args:
#         basket_rdd (list): [(key, value), ...]

#     Returns:
#         (dict): {item: hash, ...}
#     '''
#     subset_items = set()
#     for key, list_items in basket_rdd:
#         subset_items = subset_items | set(list_items)
#     subset_items = list(subset_items)
#     n = range(1, len(subset_items) + 1) # n is the number of distinct items
#     map_to_int = { item: n_item for item, n_item in zip(subset_items, n)}
#     return map_to_int

In [217]:
def hash_items(basket_rdd: list) -> dict:
    '''Translates item names into integers from 1 to n

    Args:
        basket_rdd (list): [(key, value), ...]

    Returns:
        (dict): {item: hash, ...}
    '''
    subset_items = set()
    for _, list_items in basket_rdd:
        subset_items = subset_items | set(list_items)
    subset_items = list(subset_items)
    n = range(1, len(subset_items) + 1) # n is the number of distinct items
    map_to_int = { item: n_item for item, n_item in zip(subset_items, n)}
    return map_to_int

In [522]:
basket_rdd

PythonRDD[217] at collect at /var/folders/68/6vxh_8k15_n5m7tsxrjkzsvm0000gn/T/ipykernel_33145/1166133831.py:1

In [541]:
# user_basket_memory_hashed = []
# for key, list_items in basket_rdd.collect():
#     user_basket_memory_hashedlist = [map_item_hash[item] for item in list_items]
#     user_basket_memory_key = key
#     user_basket_memory_hashed.append((user_basket_memory_key, user_basket_memory_hashedlist))

In [587]:
chunk_p_s_threshold = 3
# Hash partition data since items can be strings (higher performance)
partitionData = basket_rdd.collect()
map_item_hash = hash_items(partitionData)
inv_map_item_hash = {v: k for k, v in map_item_hash.items()} # Look-up table to convert back to original value
user_basket_memory_hashed = []
for key, list_items in partitionData:
    user_basket_memory_hashedlist = [map_item_hash[item] for item in list_items]
    user_basket_memory_key = key
    user_basket_memory_hashed.append((user_basket_memory_key, user_basket_memory_hashedlist))

# A-Pripri Algorithm 
k = 1
C_k = [(item,) for item in map_item_hash.values()] # Singletons Candidates
L_k_1 = count_itemsets_of_size_k(user_basket_memory_hashed, C_k, chunk_p_s_threshold) # all frequent singletons
L_1 = L_k_1.copy()
k += 1
while len(L_k_1) > 0:
    for freq_item in L_k_1:
        print(tuple([inv_map_item_hash[item] for item in freq_item])) # Convert back to 
    C_k = generate_candidates(L_k_1, L_1, k)
    L_k_1 = count_itemsets_of_size_k(user_basket_memory_hashed, C_k, chunk_p_s_threshold) # all frequent singletons
    k += 1
    break

('101',)
('100',)
('99',)
('105',)
('98',)
('102',)
('103',)
('97',)


In [592]:
for i in L_k_1:
    print(tuple(sorted(i)))

(3, 7)
(5, 7)
(3, 8)
(8, 12)
(2, 5)
(1, 3)
(1, 12)
(3, 12)
(2, 8)
(5, 8)
(8, 11)
(1, 2)
(1, 5)
(1, 11)
(5, 12)
(3, 5)
(1, 8)
(7, 8)
(1, 7)
(2, 12)


In [358]:
def p_s_threshold(partitionData: map, support_threshold: int, size_input_file: int) -> int:
    '''
    Lower the support threshold from s to ps if each Map task gets fraction p of the total input file
    
    Args:
        partitionData (map)
        support_threshold (int): s
        size_input_file (int)
    Returns:
        (int): p*s
    '''

    return round(support_threshold*(len(partitionData)/size_input_file))

In [593]:
def a_priori_algorithm(partitionData: map, threshold: int, size_input_file: int) -> tuple:

    partitionData = list(partitionData)
    # Compute Threshold
    chunk_p_s_threshold = p_s_threshold(partitionData, threshold, size_input_file)

    # Hash partition data since items can be strings (higher performance)
    map_item_hash = hash_items(partitionData)
    inv_map_item_hash = {v: k for k, v in map_item_hash.items()} # Look-up table to convert back to original value
    user_basket_memory_hashed = []
    for key, list_items in partitionData:
        user_basket_memory_hashedlist = [map_item_hash[item] for item in list_items]
        user_basket_memory_key = key
        user_basket_memory_hashed.append((user_basket_memory_key, user_basket_memory_hashedlist))

    # A-Pripri Algorithm 
    k = 1
    C_k = [(item,) for item in map_item_hash.values()] # Singletons Candidates
    L_k_1 = count_itemsets_of_size_k(user_basket_memory_hashed, C_k, chunk_p_s_threshold) # all frequent singletons
    L_1 = L_k_1.copy()
    k += 1
    while len(L_k_1) > 0:
        for freq_item in L_k_1:  yield (tuple([inv_map_item_hash[item] for item in tuple(sorted(freq_item))]), 1) # Convert back to 
        C_k = generate_candidates(L_k_1, L_1, k)
        L_k_1 = count_itemsets_of_size_k(user_basket_memory_hashed, C_k, chunk_p_s_threshold) # all frequent singletons
        k += 1

**Apply to rdd**

In [594]:
support_threshold = 4
size_input_file = basket_rdd.count()
map_task_1 = basket_rdd.\
    mapPartitions(lambda partition: a_priori_algorithm(partition, support_threshold, size_input_file))

In [595]:
reduce_task_1 = map_task_1.map(lambda x: x[0]).distinct().sortBy(lambda x: (len(x), x))

In [596]:
candidate_itemsets = reduce_task_1.collect()

In [597]:
def count_freq_itemsets(partitionData: map, candidate_itemsets: list) -> tuple:
    '''Count candidate itemset.The output is a set of key-value pairs (C, v),
    where C is one of the candidate sets and v is the support for that itemset among
    the baskets that were input to this Map task
    
    Args:
        partitionData (map)
    Returns:```````````````
        (tuple): (C,v)
    '''
    candidates_count = defaultdict(int)
    partitionData = list(partitionData)
    for candidate in candidate_itemsets:
        for _, item_list in partitionData:
            if set(candidate) <= set(item_list):
                candidates_count[candidate] += 1 # increase 1 count
        yield (candidate, candidates_count[candidate])

In [598]:
map_task_2 = basket_rdd\
    .mapPartitions(lambda partition: count_freq_itemsets(partition, candidate_itemsets))

In [599]:
# The Reduce tasks take the itemsets they are given as keys and sum the associated values.
# If candidate_itemset count > threshold return`` 
reduce_task_2 = map_task_2.\
    reduceByKey(lambda x,y: x + y).filter(lambda x: x[1] >= support_threshold).map(lambda x: x[0]).sortBy(lambda x: (len(x), x))

In [600]:
frequent_itemset = reduce_task_2.collect()

In [601]:
def format_output_pairs(list_of_items: list) -> str:
    '''Format list of tuple itemsets and convert them to string. Creates new line for each length of pair.

    Args:
        list_of_items (list): list of items
    Returns:
        (str):
    '''
    output_string = ''
    last_length_pair = len(list_of_items[0])
    for pair in list_of_items:
        tmp_string = "', '".join(list(pair))
        tmp_string = f"('{tmp_string}'),"
        if len(pair) != last_length_pair:
            last_length_pair = len(pair)
            output_string = output_string[:-1] + '\n' + '\n'
        output_string += tmp_string
    return output_string[:-1] # Remove last comma before \n


In [602]:
def save_to_file(file_path: str, header_candidate: str, list_of_candidates: list, header_frequent_items: str, list_of_freq_items: list) -> None:
    '''Save data to file
    Args:
        file_path (str)
        header_candidate (str)
        list_of_candidates (list)
        header_frequent_items (str)
        list_of_freq_items (list)
    '''
    with open(file_path, 'w') as fp:
        fp.write(f'{header_candidate}:\n')
        output_pairs = format_output_pairs(list_of_candidates)
        fp.write(output_pairs)
        fp.write(f'\n\n{header_frequent_items}:\n')
        output_pairs = format_output_pairs(list_of_freq_items)
        fp.write(output_pairs)

In [603]:
save_to_file('text.txt', 'Candidates', candidate_itemsets, 'Frequent Itemsets', frequent_itemset)

# References

- https://sparkbyexamples.com/pyspark/pyspark-mappartitions/